In [7]:
import sys
sys.path.insert(0, r"../helper")

from model.decentlayer import *

new_cc_mode = True

In [63]:
-1 == -1.0

True

In [59]:
torch.max(torch.tensor([0, 29, 0.3]))
torch.rand(1)

tensor([0.9131])

In [62]:
1-torch.abs(torch.tensor([2]) - torch.tensor([5]))

tensor([-2])

In [51]:
ms_in_2 = [torch.tensor(0), torch.tensor(8), torch.tensor(17)]
ns_in_2 = [torch.tensor(0), torch.tensor(8), torch.tensor(17)]

model_kwargs = {
    'in_channels' : 1, # not in use yet
    'n_classes': None, # filled in the dataset
    'out_dim' :  [1, 8, 16, 32], # [1, 8, 16, 32], #[1, 16, 24, 32] # entry, decent1, decent2, decent3
    'grid_size' : 18*18,
    'criterion': torch.nn.CrossEntropyLoss(),# torch.nn.BCEWithLogitsLoss(),
    'new_cc_mode' : new_cc_mode, # connection cost number new!!
    'optimizer': "sgd", # sgd adamw
    'base_lr': 0.001,
    'min_lr' : 0.00001,
    'momentum' : 0.9,
    'lr_update' : 100,
    # decentnet
    'cc_weight': 0.5,
    'cc_metric' : 'l1', # connection cost metric (for loss) - distance metric
    'ci_metric' : 'l2', # channel importance metric (for pruning)
    'cm_metric' : 'not implemented yet', # 'count', # crossing minimisation 
    'update_every_nth_epoch' : 10, # 5
    'pretrain_epochs' : 20, # 20
    'prune_keep' : 0.97, # 0.97, # in each epoch
    'prune_keep_total' : 0.4, # this number is not exact, depends on the prune_keep value
}

decent2 = DecentLayer(ms_in=ms_in_2, ns_in=ns_in_2, n_filters=10, 
                           kernel_size=3, stride=1, padding=0, dilation=1,
                           model_kwargs=model_kwargs, 
                           layer_name='decent2',
                           ckpt_path="")




In [52]:
# =============================================================================
# compute connection cost for this layer - based on distance
# returns:
#    connection cost for the loss function
# notes:
#    currently using l2 norm, doesn't work that well
# sources:
#    adapted from BIMT: https://github.com/KindXiaoming/BIMT/blob/main/mnist_3.5.ipynb
#    https://stackoverflow.com/questions/74086766/how-to-find-total-cost-of-each-path-in-graph-using-dictionary-in-python
# nonsense?
#    i don't even know what the following comments are about ... 
#    based on previous layer (cause I only have input ms_in, n_in information)
#    mean( sum( of connection cost between this filter and all incoming filters
#    need it for loss - aka all layers, all filters together
#    need it for swapping - this layer, all filters
#    only the active ones (we need to use the indices for that)
#    for swapping i need ??
# =============================================================================

# connection cost list
cc = []

for i_f, f in enumerate(decent2.filter_list):
    
    #print("filter", i_f)
    #print(f)
    # for each filter we use the current position and all incoming positions

    #mn = torch.cat([torch.tensor(f.m_this), torch.tensor(f.n_this)])
    #print(mn.shape)
    #msns = torch.cat([torch.tensor(f.ms_in), torch.tensor(f.ns_in)]) # .transpose(1,0)
    #print(msns.shape)
    #cc.append(torch.cdist(mn.unsqueeze(dim=0), msns.transpose(1,0), 'euclidean') / 8) # number comes from 9*9 = 81 [0-8]

    mn = torch.cat([f.m_this.unsqueeze(0), f.n_this.unsqueeze(0)]).transpose(1,0)
    msns = torch.cat([f.ms_in.unsqueeze(0), f.ns_in.unsqueeze(0)]).transpose(1,0)
    #print(mn)
    #print(msns)

    # mean ( l2 norm as distance metric / normalisation term for l2 norm)
    # mean of distances
    # normalise with max=grid square root, min=0
    # mean from all non-nan values
    #
    

    if decent2.cc_metric == 'l1':
        cc.append(torch.nanmean( torch.tensor( scipy.spatial.distance.cdist(mn.detach().cpu().numpy(), msns.detach().cpu().numpy(), 'cityblock') /decent2.grid_sqrt ) ))
    elif decent2.cc_metric == 'l2':
        cc.append(torch.nanmean( torch.tensor( scipy.spatial.distance.cdist(mn.detach().cpu().numpy(), msns.detach().cpu().numpy(), 'euclidean') /decent2.grid_sqrt ) ))
    elif decent2.cc_metric == 'l2_torch':
        cc.append(torch.nanmean( torch.cdist( a=mn.float(), b=msns.float(), p=2) /decent2.grid_sqrt ))
    elif decent2.cc_metric == 'linf':
        cc.append(torch.nanmean( torch.tensor( scipy.spatial.distance.cdist(mn.detach().cpu().numpy(), msns.detach().cpu().numpy(), 'chebyshev') /decent2.grid_sqrt ) ))
    elif decent2.cc_metric == 'cos':
        cc.append(torch.nanmean( torch.tensor( scipy.spatial.distance.cdist(mn.detach().cpu().numpy(), msns.detach().cpu().numpy(), 'cosine') /decent2.grid_sqrt ) ))
    elif decent2.cc_metric == 'jac':
        cc.append(torch.nanmean( torch.tensor( scipy.spatial.distance.cdist(mn.detach().cpu().numpy(), msns.detach().cpu().numpy(), 'jaccard') /decent2.grid_sqrt ) ))
    elif decent2.cc_metric == 'cor':
        cc.append(torch.nanmean( torch.tensor( scipy.spatial.distance.cdist(mn.detach().cpu().numpy(), msns.detach().cpu().numpy(), 'correlation') /decent2.grid_sqrt ) ))
    else:
        print("DECENT WARNING: no valid cc metric chosen.")
        
    print(torch.max( torch.tensor(scipy.spatial.distance.cdist(mn.detach().cpu().numpy(), msns.detach().cpu().numpy(), 'cityblock') /decent2.grid_sqrt) ))

# mean connection cost of a layer
# mean from all non-nan values
 # , max_cc

tensor(1.5000, dtype=torch.float64)
tensor(1.1111, dtype=torch.float64)
tensor(1.3889, dtype=torch.float64)
tensor(1.3889, dtype=torch.float64)
tensor(1.3333, dtype=torch.float64)
tensor(1.1667, dtype=torch.float64)
tensor(1.3889, dtype=torch.float64)
tensor(1.0556, dtype=torch.float64)
tensor(1.1111, dtype=torch.float64)
tensor(1.7222, dtype=torch.float64)


In [41]:
torch.nanmean(torch.tensor(cc))

tensor(0.6162, dtype=torch.float64)

In [42]:
f # a filter

DecentFilter(weights: torch.Size([1, 3, 3, 3]) at position: m_this=Parameter containing:
tensor([12.]), n_this=Parameter containing:
tensor([12.]))
 with inputs: ms_in= 0, 8, 17, ns_in= 0, 8, 17)

In [43]:
scipy.spatial.distance.cdist(mn.detach().cpu().numpy(), msns.detach().cpu().numpy(), 'euclidean')

array([[16.97056275,  5.65685425,  7.07106781]])

In [44]:
scipy.spatial.distance.cdist(mn.detach().cpu().numpy(), msns.detach().cpu().numpy(), 'euclidean') / decent2.grid_sqrt

array([[0.94280904, 0.31426968, 0.3928371 ]])

In [45]:
torch.nanmean( torch.tensor( scipy.spatial.distance.cdist(mn.detach().cpu().numpy(), msns.detach().cpu().numpy(), 'euclidean') / decent2.grid_sqrt ) )

tensor(0.5500, dtype=torch.float64)

In [46]:
torch.max( torch.tensor(scipy.spatial.distance.cdist(mn.detach().cpu().numpy(), msns.detach().cpu().numpy(), 'euclidean') / decent2.grid_sqrt) )

tensor(0.9428, dtype=torch.float64)

In [53]:
5/0.9

5.555555555555555

In [54]:
0/0.9

0.0

In [55]:
0.1/0.9

0.11111111111111112